# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [11]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import json
import matplotlib.pyplot as plt 

# Import API key
from api_keys import geoapify_key

In [12]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,edinburgh of the seven seas,-37.0676,-12.3116,16.25,74,29,9.68,SH,1706998914
1,1,grytviken,-54.2811,-36.5092,9.04,94,100,5.07,GS,1706998915
2,2,palencia,42.4167,-4.5000,6.00,68,11,0.97,ES,1706998915
3,3,orchomenos,38.4929,22.9796,4.99,73,6,1.28,GR,1706998915
4,4,talnakh,69.4865,88.3972,-14.33,96,100,3.67,RU,1706998916


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [4]:
%%capture --no-display

# Configure the map plot
city_map = city_data_df.hvplot.points(
"Lng",
"Lat",
geo=True,
color="City",
size="Humidity",
scale = .5,
alpha=.5,
tiles="OSM")

# Display the map
city_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [5]:
# Narrow down cities that fit criteria and drop any results with null values
vacayDf = city_data_df.loc[city_data_df["Wind Speed"] <= 5]
vacayDf = vacayDf.loc[vacayDf["Cloudiness"]==0]
vacayDf = vacayDf.loc[vacayDf["Max Temp"]>=22]

# Drop any rows with null values
vacayDf.count()
vacayDf = vacayDf.dropna(how="any")
vacayDf.count()

# Display sample data
vacayDf.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
9,9,port lincoln,-34.7333,135.8667,22.25,62,0,3.30,AU,1706998917
64,64,aransas pass,27.9095,-97.1500,23.36,73,0,4.02,US,1706998945
157,157,port elizabeth,-33.9180,25.5701,22.40,94,0,3.60,ZA,1706998789
164,164,whakatane,-37.9585,176.9854,22.10,57,0,2.24,NZ,1706998983
194,194,korhogo,9.4500,-5.6333,26.10,18,0,2.30,CI,1706998996


### Step 3: Create a new DataFrame called `hotel_df`.

In [6]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = vacayDf[["City","Country","Lat","Lng","Humidity"]]

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"]=""
# Display sample data
hotel_df.head()

C:\Users\tloch\AppData\Local\Temp\ipykernel_23752\2245795925.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df["Hotel Name"]=""


,City,Country,Lat,Lng,Humidity,Hotel Name
9,port lincoln,AU,-34.7333,135.8667,62,
64,aransas pass,US,27.9095,-97.1500,73,
157,port elizabeth,ZA,-33.9180,25.5701,94,
164,whakatane,NZ,-37.9585,176.9854,57,
194,korhogo,CI,9.4500,-5.6333,18,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [7]:
# Set parameters to search for a hotel
radius = 10000

params = {
    "categories":"accommodation.hotel",
    'limit':20,
    "apiKey":{geoapify_key}   
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    long = row["Lng"]
    lat = row["Lat"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{long},{lat},{radius}"
    params["bias"] = f'proximity:{long},{lat}'
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places?"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url,params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
port lincoln - nearest hotel: Boston Hotel
aransas pass - nearest hotel: Executive Inn & Suites
port elizabeth - nearest hotel: Waterford Hotel
whakatane - nearest hotel: Whakatane Hotel
korhogo - nearest hotel: Hôtel Tchimbota
cockburn town - nearest hotel: The Salt Raker Inn
freetown - nearest hotel: Formal Leona hotel
kouroussa - nearest hotel: No hotel found
ferkessedougou - nearest hotel: Ferké Hôtel Iroko
sananduva - nearest hotel: No hotel found
la penita de jaltomba - nearest hotel: Hotel PeñaMar


,City,Country,Lat,Lng,Humidity,Hotel Name
9,port lincoln,AU,-34.7333,135.8667,62,Boston Hotel
64,aransas pass,US,27.9095,-97.1500,73,Executive Inn & Suites
157,port elizabeth,ZA,-33.9180,25.5701,94,Waterford Hotel
164,whakatane,NZ,-37.9585,176.9854,57,Whakatane Hotel
194,korhogo,CI,9.4500,-5.6333,18,Hôtel Tchimbota
274,cockburn town,TC,21.4612,-71.1419,61,The Salt Raker Inn
374,freetown,SL,8.4840,-13.2299,78,Formal Leona hotel
397,kouroussa,GN,10.6500,-9.8833,18,No hotel found
449,ferkessedougou,CI,9.6000,-5.2000,17,Ferké Hôtel Iroko
488,sananduva,BR,-27.9497,-51.8067,82,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [8]:
hotel_df = hotel_df.loc[hotel_df["Hotel Name"]!="No hotel found"]
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
9,port lincoln,AU,-34.7333,135.8667,62,Boston Hotel
64,aransas pass,US,27.9095,-97.1500,73,Executive Inn & Suites
157,port elizabeth,ZA,-33.9180,25.5701,94,Waterford Hotel
164,whakatane,NZ,-37.9585,176.9854,57,Whakatane Hotel
194,korhogo,CI,9.4500,-5.6333,18,Hôtel Tchimbota
274,cockburn town,TC,21.4612,-71.1419,61,The Salt Raker Inn
374,freetown,SL,8.4840,-13.2299,78,Formal Leona hotel
449,ferkessedougou,CI,9.6000,-5.2000,17,Ferké Hôtel Iroko
521,la penita de jaltomba,MX,21.0333,-105.2500,65,Hotel PeñaMar


In [14]:
%%capture --no-display

# Configure the map plot
hotel_map = hotel_df.hvplot.points(
"Lng",
"Lat",
hover_cols = ["Hotel Name", "Country"],
geo=True,
color="City",
size="Humidity",
scale = .95,
alpha=.95,
tiles="OSM",
    frame_width = 800,
    frame_height = 600
)

# Display the map

plt.savefig("output_data/Vac.png")
hotel_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)

<Figure size 640x480 with 0 Axes>